In [1]:
import os
from tqdm import tqdm

import numpy as np
import matplotlib.pyplot as plt

import torch
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import GradScaler
from torch.cuda.amp import autocast

from torch.utils.tensorboard import SummaryWriter

import torchvision.transforms as T

import albumentations as A
from albumentations.pytorch import ToTensorV2
from albumentations import ImageOnlyTransform

import timm

import segmentation_models_pytorch as smp
from segmentation_models_pytorch.losses import DiceLoss
from segmentation_models_pytorch.losses import SoftBCEWithLogitsLoss
from torchvision.transforms.functional import hflip, vflip, rotate



from utils.metrics import dice_avg

In [2]:
class CFG:
    valid_loss_th = DiceLoss(mode='binary', from_logits=False)

In [3]:
def get_ths(all_preds, all_masks):
    all_preds_sigmoid = all_preds#torch.nn.Sigmoid()(all_preds)
    
    ths, losses = [], []
    for th in np.linspace(0.05, 1.01, 55):
        all_preds_cur = (all_preds_sigmoid > th).to(int)
        cur_loss = CFG.valid_loss_th(all_preds_cur, all_masks)
        
        ths.append(th)
        losses.append(cur_loss.item())
        
    return ths, losses

In [4]:
all_preds = torch.load('./all_preds.pth', map_location=torch.device('cpu'))

In [5]:
img_pred_sigmoid = torch.sigmoid(all_preds['full_preds']).to('cuda:0')
mask = all_preds['mask'].to('cuda:0')

In [10]:
img_pred_sigmoid_mean = (img_pred_sigmoid[:, 3, :, :] + 
                         img_pred_sigmoid[:, 4, :, :])/2.0

#img_pred_sigmoid_mean = (img_pred_sigmoid[:, 1, :, :] + 
#                         img_pred_sigmoid[:, 3, :, :] + 
#                         img_pred_sigmoid[:, 4, :, :] + 
#                         img_pred_sigmoid[:, 5, :, :])/4.0

In [11]:
with torch.no_grad():
    ths, losses_ = get_ths(img_pred_sigmoid_mean, mask)

In [12]:
np.min(losses_)

0.30913686752319336

In [13]:
ths[np.argmin(losses_)]

0.4588888888888889